In [1]:
from langchain_community.graphs import Neo4jGraph
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains import GraphCypherQAChain
import warnings
import os
warnings.filterwarnings("ignore")

In [2]:
load_dotenv()

True

In [3]:
openai_api_key = os.environ["OPENAI_API_KEY"]

Add Neo4j credentials

In [4]:
NEO4J_URL = os.environ["NEO4J_URL"]
NEO4J_USERNAME = os.environ["NEO4J_USERNAME"]
NEO4J_PASSWORD = os.environ["NEO4J_PASSWORD"]
NEO4J_DATABASE = os.environ["NEO4J_DATABASE"]

In [5]:
graph = Neo4jGraph(url=NEO4J_URL, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

Print the graph database schema

In [6]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Street {name: STRING}
City {name: STRING}
House {condition: INTEGER, sqft_basement: INTEGER, yr_built: INTEGER, yr_renovated: INTEGER, sqft_living: INTEGER, sqft_lot: INTEGER, floors: FLOAT, id: STRING, price: INTEGER, bedrooms: INTEGER, baths: INTEGER}
Relationship properties:

The relationships:
(:House)-[:LOCATED_IN]->(:City)
(:House)-[:LOCATED_ON]->(:Street)


Questions

In [7]:
q_one = "I want six houses with 1 bedroom"
q_two = "I want a house with price around 500000 and it should have 3 bedrooms"
q_three = "Give me hoses that has been built in 2001, have 4 bedrooms and in a overall good condition"
q_four = "Give me hoses that has been built after 2001, have 4 bedrooms and in a great condition, and have 1930 square feet in total"
q_five = "I want a house with price greater than 300000 and it should be located in a Kent city"
q_six = "Give me hoses that has been built after 2001 and in a great condition"

Chain

Simple Agent (a):

In [8]:
model_name = "gpt-4o-mini"
llm = ChatOpenAI(model=model_name, temperature=0)

In [9]:
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)

In [10]:
response = chain.invoke({"query": q_one})
print(response)
print("\nLLM response:", response["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (h:House) 
WHERE h.bedrooms = 1 
RETURN h 
LIMIT 6

Full Context:
[{'h': {'bedrooms': 1, 'condition': 2, 'sqft_basement': 280, 'sqft_living': 1100, 'baths': 1, 'floors': 1.0, 'yr_built': 1947, 'price': 250000, 'sqft_lot': 4373, 'id': '260', 'yr_renovated': 0}}, {'h': {'bedrooms': 1, 'condition': 5, 'sqft_basement': 0, 'sqft_living': 810, 'baths': 1, 'floors': 1.0, 'yr_built': 1941, 'price': 235000, 'sqft_lot': 2451, 'id': '477', 'yr_renovated': 0}}, {'h': {'bedrooms': 1, 'condition': 3, 'sqft_basement': 0, 'sqft_living': 700, 'baths': 1, 'floors': 1.0, 'yr_built': 1942, 'price': 350000, 'sqft_lot': 5100, 'id': '784', 'yr_renovated': 1999}}, {'h': {'bedrooms': 1, 'condition': 3, 'sqft_basement': 0, 'sqft_living': 1140, 'baths': 1, 'floors': 1.0, 'yr_built': 1920, 'price': 540000, 'sqft_lot': 6700, 'id': '831', 'yr_renovated': 1979}}, {'h': {'bedrooms': 1, 'condition': 4, 'sqft_basement': 0, 'sqft_living': 810, '

In [11]:
response = chain.invoke({"query": q_two})
print(response)
print("\nLLM response:", response["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (h:House)
WHERE h.price >= 490000 AND h.price <= 510000 AND h.bedrooms = 3
RETURN h

Full Context:
[{'h': {'bedrooms': 3, 'condition': 3, 'sqft_basement': 0, 'sqft_living': 1790, 'baths': 3, 'floors': 2.0, 'yr_built': 2010, 'price': 509900, 'sqft_lot': 2700, 'id': '138', 'yr_renovated': 0}}, {'h': {'bedrooms': 3, 'condition': 3, 'sqft_basement': 0, 'sqft_living': 1480, 'baths': 3, 'floors': 3.0, 'yr_built': 2006, 'price': 500000, 'sqft_lot': 1171, 'id': '222', 'yr_renovated': 0}}, {'h': {'bedrooms': 3, 'condition': 3, 'sqft_basement': 0, 'sqft_living': 990, 'baths': 1, 'floors': 1.0, 'yr_built': 1943, 'price': 495000, 'sqft_lot': 6000, 'id': '231', 'yr_renovated': 2002}}, {'h': {'bedrooms': 3, 'condition': 5, 'sqft_basement': 330, 'sqft_living': 1540, 'baths': 3, 'floors': 1.0, 'yr_built': 1980, 'price': 503000, 'sqft_lot': 6760, 'id': '235', 'yr_renovated': 0}}, {'h': {'bedrooms': 3, 'condition': 4, 'sqft_base

In [12]:
response = chain.invoke({"query": q_three})
print(response)
print("\nLLM response:", response["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (h:House)
WHERE h.yr_built = 2001 AND h.bedrooms = 4 AND h.condition > 3
RETURN h

Full Context:
[{'h': {'bedrooms': 4, 'condition': 4, 'sqft_basement': 0, 'sqft_living': 2280, 'baths': 3, 'floors': 2.0, 'yr_built': 2001, 'price': 433000, 'sqft_lot': 7568, 'id': '818', 'yr_renovated': 0}}]

> Finished chain.
{'query': 'Give me hoses that has been built in 2001, have 4 bedrooms and in a overall good condition', 'result': 'I have a house that was built in 2001, has 4 bedrooms, and is in overall good condition. It features 3 baths, 2 floors, a living area of 2,280 sqft, and a lot size of 7,568 sqft, with a price of $433,000.'}

LLM response: I have a house that was built in 2001, has 4 bedrooms, and is in overall good condition. It features 3 baths, 2 floors, a living area of 2,280 sqft, and a lot size of 7,568 sqft, with a price of $433,000.


Improved Agents: Contains 4 steps

1. Detecting entities in the user input

2. Match entities to database

3. Define a custom Cypher prompt that takes the entity mapping information along with the schema and the user question to construct a Cypher statement

4. Generating answers based on database results

Strategies to improve graph database query generation by mapping values from user inputs to database

When using the built-in graph chains, the LLM is aware of the graph schema, but has no information about the values of properties stored in the database. Therefore, we can introduce a new step in graph database Q&A system to accurately map values.

Detecting entities in the user input

We have to extract te types of entities/values we want to map to a graph database. In this example, we ar dealing with a movies graph, so we can map movies and people to the database.

In [13]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)

In [14]:
# system_message = """
# HomeHapp is an AI-powered assistant to help users find their ideal homes. By engaging in natural conversations, HomeHapp aims to understand users' lifestyle preferences, living spaces, and housing requirements, providing valuable insights and recommendations to make the house-hunting process more efficient and enjoyable. With extensive access to prime and super-prime London properties, including off-market, unlisted properties, HomeHapp can quickly identify homes that closely match users' visions and requirements.
# 
# Key Features
# 1. Understands and responds to user queries related to finding the property they are looking for.
# 2. Offers personalized advice and recommendations based on user preferences
# 3. Generates human-like text for natural, engaging conversations
# 4. Focuses on helping users express and define their criteria and preferences for property selection.
# 5. Currently serves USA homes priced between 100.000 to 3 million. HomeHapp doesn't currently serve properties outside of USA
# 
# Key guidelines for HomeHapp:
# 
# Search bot should have following steps of interaction with a user:
# 
# 1.1 Collect information about users desired properties
# 1.2 Prioritize understanding users' specific requirements and preferences
# 1.3 Maintain a friendly, professional tone throughout the conversation
# 1.4 Use native knowledge to answer general property and estate-related questions
# 1.5 Ask for missing information, encourage detailed preferences, and summarize requirements
# 1.7 Keep users informed about the search process and set expectations for the results
# 
# Conversation Flow
# 1. Greet the user inquire about their property requirements and preferences
# 2. Ask for missing information
# 3. Summarize and confirm user requirements
# 
# 1.1 Conversation Flow ( prompts )
# 
# 1. Greeting: The bot begins by warmly greeting the user, engaging them in a friendly and conversational manner.
#  Initial Inquiry: It then inquires about the type of property the user is looking for, encouraging them to share both key requirements and personal preferences that would make the property ideal for their lifestyle.
# Available attributes that the bot should consider asking are Price of the house, Number of bedrooms, Number of bathrooms, Square footage of living space, Total area of the land, Number of floors, Condition, Area of the basement, Year Built, Year Renovated, City and the number of houses.
# 2. Ask for missing information: The bot swiftly inquires about any missing features from this list:
# Available attributes that the bot should consider asking are Price of the house, Number of bedrooms, Number of bathrooms, Square footage of living space, Total area of the land, Number of floors, Condition, Area of the basement, Year Built, Year Renovated, City and the number of houses.
# 3. Summarization and Confirmation: The bot summarizes the user's requirements and asks for confirmation.
# 
# 1.2 Conversation Flow example:
# 
# - User: "I'm looking for a three-bedroom house in Seattle. I would like the house to have 3 bedrooms and 2 bathrooms."
# - HomeHapp: "Great! To help narrow down the search, could you please provide your budget for this apartment?"
# - User: "I'm thinking around 500k."
# - HomeHapp: "Thank you for the information. To further refine the search, are there any other essential property features 
# like square footage of living space, total area of the land, number of floors, condition, area of the basement, Year Built, or Year Renovated you're looking for?"
# - User: "I would like the living area to be more around 1900 sqft."
# - HomeHapp: "Noted. Let me summarize your requirements:
#   - Three-bedroom house in Seattle
#   - Budget: 500.000
#   - Number of bedrooms: 3
#   - Number of bathrooms: 2
#   - Square footage of living space: 1900 sqft
# 
#   I'll begin the search now. I'll present the best matches shortly."
# 
# 
# 2. Frequently asked questions and suggested answers:
# 
# Q: Who are you and what can you do?
# A: I'm a ChatGPT-4 powered AI property search assistant. I can help you find best suited prime and super prime properties in London, offer insights into the real estate market, and guide you through the property buying process.
# 
# Q: Can you help me find properties outside USA?
# A: Currently, my primary focus is on the USA market.
# 
# Q: Do you speak any language other than English?
# A: Currently, I operate primarily in English. However, future updates may include multilingual support to cater to a wider range of users.
# 
# Q: Can you tell me how to use this app?
# A: While I can offer general guidance on real estate apps, specific instructions for using the HomeHap app, including features and navigation, would typically be found within the app's help section or user manual.
# 
# For Developer:
# 
# - After the bot summarizes the user's hard criteria, store these criteria in a dictionary. The dictionary should include keys such as 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'condition', 'sqft_basement', 'yr_built', and 'yr_renovated'.
# 
# - Implement a flag, `information_complete`, initially set to `False`. This flag should be set to `True` once the bot has gathered all necessary information from the user and has made the summarization.
# 
# Example:
# 
# ```python
# user_criteria = {
#     "price": None,
#     "bedrooms": None,
#     "bathrooms": None,
#     "sqft_living": None,
#     "sqft_lot": None,
#     "floors": None,
#     "condition": None,
#     "sqft_basement": None,
#     "yr_built": None,
#     "yr_renovated": None,
#     "city": None,
#     "limit": None
# }
# 
# information_complete = False
# 
# # Update user_criteria based on user input, and set information_complete to True when summarization was made.
# """

In [15]:
# def generate_final_output(llm_role):
#     # User prompt
#     user_prompt = mapped_attributes
#     
#     # Create the chat completion
#     llm_response = client.chat.completions.create(
#         model="gpt-4o-mini",  # Specify the model you want to use
#         messages=[
#             {"role": "system", "content": llm_role},
#             {"role": "user", "content": user_prompt}
#         ]
#     )
#     
#     return llm_response.choices[0].message.content

In [16]:
print(q_three)

Give me hoses that has been built in 2001, have 4 bedrooms and in a overall good condition


In [17]:
# System role definition
system_role = (
    """
    You are given a question about house preferences. Your task is to extract specific attributes and their corresponding values based on the user's requirements. Ignore explainable words such as "location" when it relates to city and street and "dollars" or "usd" when it relates to price. Additionally, convert any qualitative descriptions like "great" or "excellent" into a numerical value based on a scale of 1-5. Return the information as a dictionary where the keys are the column names from the database and the values are the user's specified numbers.

    Guidelines for Attribute Extraction:
        price:
            Extract numerical values directly related to the house's price.
            Assign to `max_price` if the context implies an upper bound (e.g., "less than 7 million", "up to 200000").
            Assign to `min_price` if the context implies a lower bound (e.g., "more than 500000 dollars").
            Handle cases where prices are mentioned in terms like "thousands," "500k," or "millions" by converting them to numerical values (e.g., 500k to 500000, 7 million to 7000000).
            You can not assign more than one of these price attributes.
        bedrooms: Extract the number of bedrooms.
        bathrooms: Extract the number of bathrooms.
        Square Footage:
            Assign to sqft_living if it refers to the total interior living space.
            Assign to sqft_lot if it refers to the total land area.
            Assign to sqft_basement if the context specifically mentions basement square footage.
        floors: Extract the number of floors in the house.
        condition: Convert qualitative descriptions of condition into numerical values:
            "Poor" = 1
            "Fair" = 2
            "Good" = 3
            "Very Good" = 4
            "Great" or "Excellent" = 5
        year:
            Extract numerical values directly related to the house's year of construction or renovation.
            Assign to `yr_built` if the context clearly indicates the original construction of the house (e.g., "built in 1995", "constructed in 2002").
            Assign to `yr_renovated` if the context suggests any renovation, update, or refurbishment year (e.g., "renovated in 2010", "updated in 2015").
            Look for keywords such as "built," "constructed," "originally," "new," or "first built" to determine `yr_built`.
            Identify terms like "renovated," "updated," "modernized," "refurbished," or "redone" to assign to `yr_renovated`.
            Give precedence to `yr_renovated` over `yr_built` if both are mentioned in the same context, as renovation is often a newer event.
            Ensure that each year is only assigned to one category and does not overlap.
        street:
            Extract street names into the street column.
        city:
            Extract city names into the city column.
        limit:
            Extract the number if the user specifies a limit to the number of houses they want (e.g., "show me 5 houses", "I want to see 10 listings").
            Ensure that the number is extracted and assigned to the `limit` attribute.
        number conversion:
            Convert numbers written as words into digits (e.g., "four" to 4, "five" to 5).
    """
)

In [18]:
import json
import re

In [19]:
def extract_attributes(llm_role, question):
    # User prompt
    user_prompt = (
        "### User Prompt ###\n"
        f"{question}\n"
        "### Response ###\n"
        "Please respond with a dictionary.\n"
    )
    
    # Create the chat completion
    llm_response = client.chat.completions.create(
        model="gpt-4o-mini",  # Specify the model you want to use
        messages=[
            {"role": "system", "content": llm_role},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    response_content = llm_response.choices[0].message.content

    # Remove any code block formatting
    response_content = response_content.strip("```python").strip("```").strip()

    # Try to parse the response content as JSON
    try:
        attributes_dict = json.loads(response_content)
    except json.JSONDecodeError:
        print(f"Error decoding JSON. Attempting to parse manually.")
        attributes_dict = {}

        # Use regex to capture dictionary format if it's not a valid JSON
        match = re.search(r'{[^}]*}', response_content)
        if match:
            try:
                attributes_dict = eval(match.group())
            except Exception as e:
                print(f"Error evaluating the dictionary: {e}")
    
    return attributes_dict



In [20]:
for i in range(10):
    print(f"Response {i+1}: {extract_attributes(system_role, q_one)}")

Response 1: {'limit': 6, 'bedrooms': 1}
Response 2: {'limit': 6, 'bedrooms': 1}
Response 3: {'limit': 6, 'bedrooms': 1}
Response 4: {'limit': 6, 'bedrooms': 1}
Response 5: {'limit': 6, 'bedrooms': 1}
Response 6: {'limit': 6, 'bedrooms': 1}
Response 7: {'limit': 6, 'bedrooms': 1}
Response 8: {'limit': 6, 'bedrooms': 1}
Response 9: {'limit': 6, 'bedrooms': 1}
Response 10: {'limit': 6, 'bedrooms': 1}


In [21]:
from typing import Dict

In [28]:
def map_to_database(values: Dict)->str:
    """
    Maps the values to entities in the database and returns the mapping information.

    :param values: A dict of values to map entities in database.
    :return: A string containing the mapping information of each value to entities in the database.
    """
    
    # Extract parameters directly from the input values
    min_price = values.get('min_price', None) 
    max_price = values.get('max_price', None)
    bedrooms = values.get('bedrooms', None)
    baths = values.get('baths', None)
    sqft_living = values.get('sqft_living', None)
    sqft_lot = values.get('sqft_lot', None)
    floors = values.get('floors', None)
    condition = values.get('condition', None)
    sqft_basement = values.get('sqft_basement', None)
    yr_built = values.get('yr_built', None)
    yr_renovated = values.get('yr_renovated', None)
    street = values.get('street', None)
    city = values.get('city', None)
    limit = values.get('limit', 5)

    # Log the actual parameters being used
    print(f"min_price: {min_price}, max_price: {max_price}, bedrooms: {bedrooms}, "
          f"baths: {baths}, sqftLiving: {sqft_living}, sqftLot: {sqft_lot}, "
          f"floors: {floors}, condition: {condition}, sqftBasement: {sqft_basement}, "
          f"yrBuilt: {yr_built}, yrRenovated: {yr_renovated}, city: {city}, "
          f"street: {street}, limit: {limit}")
    
    match_query = """
    MATCH (h:House)
    WHERE 
        ($min_price IS NULL OR h.price >= $min_price) AND
        ($max_price IS NULL OR h.price <= $max_price AND h.price >= $max_price - 100.000) AND
        ($bedrooms IS NULL OR h.bedrooms = $bedrooms) AND
        ($baths IS NULL OR h.baths = $baths) AND
        ($sqftLiving IS NULL OR h.sqft_living >= $sqftLiving) AND
        ($sqftLot IS NULL OR h.sqft_lot >= $sqftLot) AND
        ($floors IS NULL OR h.floors = $floors) AND
        ($condition IS NULL OR h.condition = $condition) AND
        ($sqftBasement IS NULL OR h.sqft_basement = $sqftBasement) AND
        ($yrBuilt IS NULL OR h.yr_built >= $yrBuilt) AND
        ($yrRenovated IS NULL OR h.yr_renovated >= $yrRenovated)
    MATCH (h)-[:LOCATED_ON]->(s:Street)
    WHERE 
        ($street IS NULL OR s.name = $street)
    MATCH (h)-[:LOCATED_IN]->(c:City)
    WHERE 
        ($city IS NULL OR c.name = $city)
    RETURN h, s, c
    LIMIT $limit
    """
    wrong_one = {'min_price': None, 'max_price': None, 'bedrooms': None, 'baths': None, 'sqftLiving': None, 'sqftLot': None, 'floors': 1, 'condition': None, 'sqftBasement': None, 'yrBuilt': None, 'yrRenovated': None, 'street': None, 'city': None, 'limit': None}
    
    right_one = {'min_price': None, 'max_price': None, 'bedrooms': None, 'baths': None, 'sqftLiving': None, 'sqftLot': None, 'floors': 1, 'condition': None, 'sqftBasement': None, 'yrBuilt': None, 'yrRenovated': None, 'street': None, 'city': None, 'limit': 5}
    # query_response = graph.query(match_query, wrong_one)
    right_one = {
            "min_price": min_price,
            "max_price": max_price,
            "bedrooms": bedrooms,
            "baths": baths,
            "sqftLiving": sqft_living,
            "sqftLot": sqft_lot,
            "floors": floors,
            "condition": condition,
            "sqftBasement": sqft_basement,
            "yrBuilt": yr_built,
            "yrRenovated": yr_renovated,
            "street": street,
            "city": city,
            "limit": limit
        }
    
    print(right_one)

    # Query the database with parameters
    query_response = graph.query(
        match_query, right_one
    )

    # Process each result
    result = ""

    if query_response:
        for record in query_response:
            house = record.get('h', None)
            street_info = record.get('s', None)
            city_info = record.get('c', None)

            street_name = street_info['name'] if street_info else "N/A"
            city_name = city_info['name'] if city_info else "N/A"

            result += (
                f"House found: Price of the house: {house['price']}, Number of bedrooms: {house['bedrooms']}, "
                f"Number of bathrooms: {house['baths']}, Square footage of living space: {house['sqft_living']}, "
                f"Total area of the land : {house['sqft_lot']} ft, Number of floors: {int(house['floors'])}, "
                f"Condition: {house['condition']}, Area of the basement: {house['sqft_basement']} ft, "
                f"Year Built: {house['yr_built']}, Year Renovated: {house['yr_renovated']}, "
                f"Street: {street_name}, City: {city_name}\n"
            )
    else:
        result += "No matching house found in the database.\n"
    return result

In [29]:
test_question = "I want a house with 1 floor"

In [30]:
question = test_question
extracted_attributes = extract_attributes(system_role, question)
mapped_attributes = map_to_database(extracted_attributes)
print(question)
print(extracted_attributes)
print(mapped_attributes)

min_price: None, max_price: None, bedrooms: None, baths: None, sqftLiving: None, sqftLot: None, floors: 1, condition: None, sqftBasement: None, yrBuilt: None, yrRenovated: None, city: None, street: None, limit: 5
{'min_price': None, 'max_price': None, 'bedrooms': None, 'baths': None, 'sqftLiving': None, 'sqftLot': None, 'floors': 1, 'condition': None, 'sqftBasement': None, 'yrBuilt': None, 'yrRenovated': None, 'street': None, 'city': None, 'limit': 5}
I want a house with 1 floor
{'floors': 1}
House found: Price of the house: 805000, Number of bedrooms: 5, Number of bathrooms: 3, Square footage of living space: 4600, Total area of the land : 19831 ft, Number of floors: 1, Condition: 3, Area of the basement: 2300 ft, Year Built: 1956, Year Renovated: 2013, Street: 18015 15th Ave NW, City: Shoreline
House found: Price of the house: 275000, Number of bedrooms: 2, Number of bathrooms: 1, Square footage of living space: 1180, Total area of the land : 6552 ft, Number of floors: 1, Condition: 

Final output

In [ ]:
system_message = """
You are an AI assistant designed to present information about houses in a user-friendly format. Your task is to reformat a list of house details into a clear and structured format that enhances readability and understanding.

Each house listing should be presented with key details highlighted in a way that users can quickly grasp important information. Use bullet points or other formatting techniques to separate different attributes.

When certain attributes have a value of 0, such as "Area of the basement" or "Year Renovated," provide additional explanations to the user:
- If "Area of the basement" is 0, indicate that the house does not have a basement.
- If "Year Renovated" is 0, indicate that the house has not been renovated.

Ensure that the information is consistent and easy to read, maintaining a professional and approachable tone. Here is how each house listing should be structured:

- **Price**: [Formatted Price]
- **Number of Bedrooms**: [Bedrooms]
- **Number of Bathrooms**: [Bathrooms]
- **Square Footage of Living Space**: [Sqft Living]
- **Total Area of the Land**: [Sqft Lot]
- **Number of Floors**: [Floors]
- **Condition**: [Condition Rating]
- **Area of the Basement**: [Sqft Basement] (or indicate "No Basement" if 0)
- **Year Built**: [Yr Built]
- **Year Renovated**: [Yr Renovated] (or indicate "Not Renovated" if 0)
- **Street**: [Street Name]
- **City**: [City Name]

Add any additional formatting that improves clarity and user engagement.
"""

In [ ]:
def generate_final_output(llm_role):
    # User prompt
    user_prompt = mapped_attributes
    
    # Create the chat completion
    llm_response = client.chat.completions.create(
        model="gpt-4o-mini",  # Specify the model you want to use
        messages=[
            {"role": "system", "content": llm_role},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return llm_response.choices[0].message.content

In [ ]:
print(generate_final_output(system_message))